In [3]:
import numpy as np
import findspark
findspark.init()
import pyspark
from pyspark import sql
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql import Window
from pyspark.sql.types import IntegerType, DoubleType
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
spark = sql.SparkSession.builder \
        .master('local[*]') \
        .appName("local-spark-session") \
        .getOrCreate()

In [4]:
df = pd.DataFrame(np.random.randint(0,100,size=(15, 4)), columns=list('ABCD'))
x = df.drop('D', axis=1)
y = df['D']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
model = LogisticRegression(random_state=0).fit(x_train, y_train)
model.predict(x_test)

C:\Users\Сергей\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([39, 49, 59])

In [6]:
spark_df = spark.createDataFrame(x_test)
spark_df.show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
| 40| 75| 93|
| 31| 25| 89|
| 45| 83| 80|
+---+---+---+



In [7]:
features = ['A', 'B', 'C']

In [8]:
@pandas_udf(returnType=DoubleType())
def predict_udf(*cols):
    X = pd.concat(cols, axis=1)
    return pd.Series(model.predict(X))

In [9]:
spark_df = spark_df.withColumn('predict', predict_udf(*features))

In [10]:
spark_df.show()

+---+---+---+-------+
|  A|  B|  C|predict|
+---+---+---+-------+
| 40| 75| 93|   39.0|
| 31| 25| 89|   49.0|
| 45| 83| 80|   59.0|
+---+---+---+-------+



In [11]:
spark.stop()